In [5]:
import os
import json
import spacy
from spacy.util import filter_spans
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
import os
import json

# Function to process a single JSON file
def process_json_file(file_path):
    # Load the JSON data from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Initialize variables to store tokens and entities
    tokens = data['tokens']
    entities = data['entities']

    # Initialize dictionary to store the formatted data
    formatted_data = {'text': [], 'entities': []}

    # Iterate through tokens to reconstruct the text
    text = ''
    for token in tokens:
        text += token[2] + ' '

    # Add the reconstructed text to the formatted data
    formatted_data['text'].append(text.strip())

    # Iterate through entities to reformat and add them to the formatted data
    for entity in entities:
        formatted_entity = {'text': entity[0], 'type': entity[1], 'start': entity[2], 'end': entity[3]}
        formatted_data['entities'].append(formatted_entity)

    # Write the formatted data to a new JSON file
    output_file_path = os.path.splitext(file_path)[0] + '_output.json'
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(formatted_data, outfile, ensure_ascii=False, indent=4)

# Path to the folder containing the JSON files
folder_path = 'D:\\KnoGr\\NEWDATA\\arner\\JSONTRIANTEST'

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        # Process each JSON file
        file_path = os.path.join(folder_path, filename)
        process_json_file(file_path)

In [10]:
import os
import json

training_data = {'classes': ['BANK', 'ORG', 'PERSON', 'OFFICIAL', 'NATIONALITY', 'COUNTRY', 'MEDIA', 'FINANCIAL_INSTRUMENT', 'TIME', 'QUNATITY_OR_UNIT', 'GOVERNMENT_ENTITY', 'CORP', 'PRODUCT_OR_SERVICE', 'STOCK_EXCHANGE', 'CURRENCY', 'ROLE', 'GPE', 'CITY', 'FinMarket', 'Metrics', 'Events'], 'annotations': []}

# Specify the directory where your JSON files are
directory = 'D:\\KnoGr\\NEWDATA\\arner\\JSONFINAL\\20'

# Loop over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        filepath = os.path.join(directory, filename)
        
        # Load the JSON file with explicit encoding
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Process the data as before
        for example in data['text']:
            temp_dict = {'text': example, 'entities': []}
            for entity in data['entities']:
                entity_data = {
                    'text': entity['text'],
                    'type': entity['type'],
                    'start': entity['start'],
                    'end': entity['end']
                }
                temp_dict['entities'].append(entity_data)
            training_data['annotations'].append(temp_dict)

print(json.dumps(training_data['annotations'][3], indent=4, ensure_ascii=False))

{
    "text": "محمد صدقه   شهدت صناعه المستلزمات الطبيه تطورا كبيرا في الفتره الاخيره بمعدل نمو بلغ 30 سنويا خلال السنوات الخمس الاخيره وبدات الدخول السوق الاوروبيه ومنافسه المنتجات العالميه مما يتطلب تحقيق المزيد من الاستثمارات الجديده للتوسع في التصنيع والتصدير خاصه بعد غياب الدور البنكي عن تمويل القطاع الي جانب عدم وجود شركات للقطاع العام   في هذا السياق اكد محمد اسماعيل عبده رءيس شعبه المستلزمات الطبيه رءيس شركه الرضا للمستلزمات الطبيه في حواره مع المال ان استثمارات قطاع المستلزمات الطبيه بلغت خمسه مليارات جنيه لافتا الي ان القطاع يحتاج الي دخول استثمارات جديده خاصه المصانع الضخمه لتخفيض التكاليف والاستفاده من اقتصادات الحجم الكبير بدلا من الاعتماد علي التمويل او الاقراض نظرا لارتفاع سعر الفاءده وفرض سياسات تضع المستثمرين احيانا في مازق وتحول دون نمو القطاع \n   واشار عبده الي تنوع المستلزمات الطبيه المصنعه التي وصلت الي 8500 منتج ووصل عدد المصانع الي 171 مصنعا لم يستحوذ منها القطاع العام علي اي استثمارات وبلغ نصيب الخاص منها حوالي 91 فيما نالت الاستثمارات الاجنبيه اقل من 10 من اجم

In [8]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("ar") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [11]:
# Splitting into train and dev sets
import random
docs = list(doc_bin.get_docs(nlp.vocab))
random.shuffle(docs)
val = int(0.8 * len(docs))
train_docs = docs[:val]
dev_docs = docs[val:]
train_docbin = DocBin(docs=train_docs)
train_docbin.to_disk("train.spacy")
dev_docbin = DocBin(docs=dev_docs)
dev_docbin.to_disk("dev.spacy")

In [11]:
from spacy.util import filter_spans
from tqdm import tqdm

for training_example in tqdm(training_data['annotations']): 
    try:
        text = training_example['text']
        labels = training_example['entities']
        doc = nlp.make_doc(text) 
        ents = []
        for entity in labels:
            start_word_pos = entity['start']
            end_word_pos = entity['end'] - 1  # Subtract one here
            label = entity['type']  # Assuming the entity type is stored under the key 'type'
            
            # Convert word positions to character positions
            start_char_pos = doc[start_word_pos].idx
            end_char_pos = doc[end_word_pos].idx + len(doc[end_word_pos])

            span = doc.char_span(start_char_pos, end_char_pos, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        filtered_ents = filter_spans(ents)
        doc.ents = filtered_ents 
        doc_bin.add(doc)
    except Exception as e:
        print(f"Error occurred with text: {text}")
        print(str(e))

doc_bin.to_disk("development.spacy")  # save the docbin object 


 97%|█████████▋| 6020/6231 [00:18<00:00, 211.24it/s]

Error occurred with text: اجراءات قضاءيه غير مسبوقه ضد مصرفيين ومصارف لبنانيه .. الاسباب والتداعيات 


 تتحدث معطيات عن احتمال تدخل الحكومه ، وتوجه جمعيه المصارف اللبنانيه الي التصعيد ردا علي القرارات القضاءيه ، ومن ذلك اللجوء الي اعلان الاضراب العام في القطاع المصرفي . 
  بيروت –   اتخذ القضاء اللبناني اجراءات جديده تجاه مصارف تجاريه عنوانها استرداد المودعين لاموالهم المحتجزه ، وتزامن ذلك مع صراع سياسي محموم بين مدافعين عن الاجراءات القضاءيه ورافضين لها ، وتجلت اخيرا بالحجز علي احد اكبر البنوك اللبنانيه ، وايقاف شقيق حاكم مصرف لبنان المركزي رياض سلامه . 
 وتتوالي الاجراءات القضاءيه المحليه بوتيره متسارعه بحق المصارف والمصرفيين وحاكميه مصرف لبنان ، توازيها تحركات قضاءيه دوليه استنادا الي دعاوي رفعها مودعون وجمعيات ، واخر فصولها قرار المحكمه البريطانيه بالزام مصرفين لبنانيين هما بنك عوده و سوسيتيه جنرال دفع 4 ملايين دولار لاحد المودعين البريطانيين ؛ وهو ما رد عليه بنك عوده باغلاق عشرات الحسابات العاءده للبنانيين يحملون الجنسيه البريطانيه او اقارب لهم . 
 واوقفت المدعيه العامه في جبل لبن

 98%|█████████▊| 6110/6231 [00:19<00:00, 214.81it/s]

Error occurred with text: تعرف علي انعكاسات رفع سعر الفاءده علي الاقتصاد التركي 


 الترتيب الزمني للاحداث في تركيا ، كان له دور مهم في قراءه الواقع الاقتصادي للبلاد ، فبعد انتخابات رءاسيه وبرلمانيه ، حسمت بفوز الرءيس رجب طيب اردوغان وتحالف الجمهور بزعامه حزب العداله والتنميه ، اتت اداره جديده للسياسه النقديه ، تنبا لها البعض منذ اللحظه الاولي ان تتبني نهجا يخالف ما كانت عليه الامور قبل الانتخابات . 
 تعد قضيه التضخم ضاغطه علي الحياه الاقتصاديه والسياسيه للبلاد ، مع ما يصاحبها من تراجع في قيمه العمله المحليه ، وجاءت وعود الرءيس اردوغان في حملته الانتخابيه بتحسين الواقع الاقتصادي ، عبر تقليل معدلات البطاله ، واستقرار سعر الصرف ، وتراجع معدلات التضخم ، وهي ايضا مطالب يسعي اليها شركاء النشاط الاقتصادي في تركيا ، وتمثل تحديات امام صناع السياسه الاقتصاديه بشكل عام ، وصناع السياسه النقديه بشكل خاص . 

 منذ الاعلان عن طرح اسم الاقتصادي التركي محمد شيمشك لتولي منصب وزاره الماليه ، توقع الخبراء الغربيون والعديد من الاقتصاديين ، ان تتجه مسارات السياسه النقديه لمنحي اخر ، فذهب البعض الي حدوث تغير

100%|██████████| 6231/6231 [00:19<00:00, 316.52it/s]


In [10]:
!python -m spacy init fill-config base_config.cfg config.cfg 

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


e:\PROGS\py\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
e:\PROGS\py\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train training_data.spacy --paths.dev test.spacy

In [ ]:
!spacy evaluate output/model-best/ dev.spacy

In [ ]:
from spacy import displacy 
nlp=spacy.load("output/model-best")
text = ''' نشر موقع top hotel تقريرا كشف فيه عن تخطيط قطر لافتتاح اول عقاراتها في طوكيو قبل نهايه شهر يونيو من السنه الحاليه ، وذلك بالشراكه مع مجموعه اسكوت السنغافوريه التي نجحت رفقه صندوق قطر السيادي في الحصول علي احدي الصفقات العقاريه داخل العاصمه اليابانيه في شهر اغسطس من العام الماضي ، وهما اللذان تمكنا من الحصول علي مشروع  lyf Ginza Tokyo  الضخم للفندقه والشقق السكنيه الفخمه بالكامل في شهر اغسطس من العام الماضي ، الذي كان شاهدا علي ولوج الدوحه لعالم العقارات في اليابان ، التي كانت كثيرا مكانا تتجه اليه في السابق للحصول علي حصص وصفقات في الشركات العامله في القطاع التكنولوجي.اقرا ايضاقطر للمال يسلط الضوء علي اهميه القطاع التكنولوجي في رفد الاقتصاد القطريالصين تطرح رءيتها بشان مستقبلها كقوه تكنولوجيهرءيس غرفه قطر: موازنه الدوله للعام 2024 تعكس الاهتمام بتطوير الاقتصاددور كبيرواكد التقرير الدور الكبير الذي لعبته الاستثمارات القطريه في تطوير وتنميه فندق lyf Ginza Tokyo ، الذي بات بعد عمليه التجديد والتوسعه مجهزا ب 140 غرفه قادره علي تلبيه الطلب علي مناطق الاقامه في منطقته المعروفه بتكتلاتها الكثيره بسبب تموقعها وسط العديد من الشركات الناشءه والموجوده في مكان قريب ، ما سيضع العديد من المسافرين الي طوكيو امام خيارات سكنيه جديده ، توفر كل سبل الراحه و الاستمتاع للمقيمين القادمين اليها من مختلف عواصم العالم ، او من داخل اليابان بحثا عن الاستفاده من السلع والمنتجات الاستهلاكيه والخدماتيه التي تطرحها الشركات غير البعيده عن هذا الفندق.توجه جديدوبين التقرير ان فندق lyf Ginza Tokyo ليس الاستثمار القطري الوحيد في قطاع العقارات داخل اليابان ، مشيرا الي تمكن صندوق قطر السيادي من تملك 32 مبني سكنيا في اليابان ، ضمن مجموعه من المدن في طوكيو ، بالاضافه الي اوساكا وكيوتو ويوكوهاما وسابورو ، وذلك عبر مجموعه جاو كابيتال بارتنرز الواقع مقرها في هونغ كونغ ، والتابعه له ، حيث تم ذلك علي مرحلتين نجحت في اولاهما الدوحه في الاستحواذ علي 12 بنايه ، تلتها بعشرين اخري في المرحله الثانيه التي جاءت لتءكد الاهتمام القطري بمجال العقارات في اليابان التي تعد وجهه جديده للمشاريع القطريه في هذا الجانب بالذات ، مرجعا ذلك الي الاستقرار الكبير الذي يشهده في الوقت الراهن ، وتوفره علي هامش تطور كبير خلال الفتره المستقبليه ، بالنظر الي اعتماد المختصين في هذا المجال في اليابان علي العديد من الاساليب الفريده من نوعها ، التي تستند في الكثير من الاحيان الي التكنولوجيا ، بحكم اختلاف اليابان عن الدول الاخري طبيعيا. %18 ارتفاع الصادرات التركيه في يناير'''
text = re.sub("[إأٱآا]", "ا", text)
text = re.sub("ى", "ي", text)
text = re.sub("ًٌَُ‘ٍِـ،ْ", "", text)
text = re.sub("ؤ", "ء", text)
text = re.sub("ئ", "ء", text)
text = re.sub("ة", "ه", text)
doc=nlp(text)
displacy.render(doc,style="ent",jupyter=True)